In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [3]:
os.chdir('/Users/lauosgom/Desktop/zoomcamp/')

In [24]:
import mlflow

mlflow.set_experiment(experiment_id='736462624464899721')

mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.xgboost.autolog()



In [25]:
df_train = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')
df_val = pd.read_parquet('data/yellow_tripdata_2022-02.parquet')

In [26]:
# calculate the duration of the trip
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train['duration'] = df_train['duration'].apply(lambda td: td.total_seconds() / 60)

In [27]:
df_train = df_train[(df_train['duration'] >= 1) & (df_train['duration'] <= 60) ]

In [29]:
# decide which categorical and numerical variables we are going to use
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [30]:
df_train[categorical] = df_train[categorical].astype(str)

In [31]:
train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')

In [32]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [34]:
target = 'duration'
y_train = df_train[target].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared = False)

7.001496179445599

In [24]:
with mlflow.start_run():

    mlflow.set_tag('developer', 'lospina')

    mlflow.log_param('train-data-path', 'data/yellow_tripdata_2022-01.parquet')
    mlflow.log_param('valid-data-path', 'data/yellow_tripdata_2022-02.parquet')
    
    alpha = 0.01
    mlflow.log_param('alpha', alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_train)

    rmse = mean_squared_error(y_train, y_pred, squared = False)
    mlflow.log_metric('rmse', rmse)


In [35]:
# do the validation with the february data

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val['duration'] = df_val['duration'].apply(lambda td: td.total_seconds() / 60)

df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60) ]

df_val[categorical] = df_val[categorical].astype(str)

val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')

dv = DictVectorizer()
X_val = dv.fit_transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

In [36]:
#ml flow with hyperparameter tuning

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [37]:
train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label = y_val)

In [38]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, 'validation')],
            early_stopping_rounds = 2
            #early_stopping_rounds = 50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared = False)
        mlflow.log_metric('rmse', rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [28]:
# search space - ranges in which i want hyperopt to search for the best parameters

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, 1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42,
}

best_result = fmin(
    fn = objective,
    space = search_space,
    algo = tpe.suggest,
    #max_evals = 50,
    max_evals = 10,
    trials = Trials())

[17:16:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:12.15812                          
[1]	validation-rmse:10.37061                          
[2]	validation-rmse:9.76990                           
[3]	validation-rmse:10.22620                          
[17:16:49] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:14.97048                                                    
[1]	validation-rmse:13.78830                                                    
[2]	validation-rmse:12.77728                                                    
[3]	validation-rmse:12.00567                                                    
[4]	validation-rmse:11.300

KeyboardInterrupt: 

In [39]:
#after sorting the mlflow runs by rmse, the best parameters are:
params = {
    'learning_rate':	0.204,
    'max_depth':	17,
    'min_child_weight':	1.2,
    'objective':	'reg:linear',
    'reg_alpha':	0.2856,
    'reg_lambda':	0.00426,
    'seed':	42
}

booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, 'validation')],
            early_stopping_rounds = 2
            #early_stopping_rounds = 50
        )

2023/08/18 17:34:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1830519e6fe94669a443cf03ad183458', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[17:34:55] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:13.22700
[1]	validation-rmse:11.24376
[2]	validation-rmse:10.16650
[3]	validation-rmse:9.77519
[4]	validation-rmse:9.75292
[5]	validation-rmse:10.01861
[6]	validation-rmse:10.32297
